In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.utils import algorithm_globals
from sklearn.model_selection import train_test_split

from qiskit.quantum_info import Statevector, partial_trace
import os
import sys
from qiskit import transpile
from data import load_01
import autograd.numpy as anp
import pennylane as qml


sys.path.append('..')
from q_alchemy.qiskit import QAlchemyInitialize
from q_alchemy.qiskit_to_pennylane import convert_qiskit

os.environ["Q_ALCHEMY_API_KEY"] = "n6I5ypSXJeb8E1mlX71gAJ1v9RCKSb52"

algorithm_globals.random_seed = 12345

## SU4 convolutional layer

In [2]:
def U_SU4(params, wires): # 15 params
    qml.U3(params[0], params[1], params[2], wires=wires[0])
    qml.U3(params[3], params[4], params[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[6], wires=wires[0])
    qml.RZ(params[7], wires=wires[1])
    qml.CNOT(wires=[wires[1], wires[0]])
    qml.RY(params[8], wires=wires[0])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.U3(params[9], params[10], params[11], wires=wires[0])
    qml.U3(params[12], params[13], params[14], wires=wires[1])


In [5]:
def preparation_circuit(X):
    sp_org = QAlchemyInitialize(X, opt_params={'max_fidelity_loss':0.0})
    qc = transpile(sp_org.definition, basis_gates=["id", "rx", "ry", "rz", "cx"])
    return qc

In [7]:
dev = qml.device('default.qubit', wires = 8)
@qml.qnode(dev)
def QCNN(X, U, params):
    qc = preparation_circuit(X)
    converter = convert_qiskit(qc)
    gates, state_params, qubits = converter.circuit_to_list()
    
    def prep_circuit(gates, state_params, qubits):
        for i in range(len(gates)):
            if gates[i] == 'rz':
                qml.RZ(float(state_params[i]), wires= qc.num_qubits-1-int(qubits[i][0]))
            elif gates[i] == 'cx':
                qml.CNOT(wires=[qc.num_qubits-1-int(qubits[i][0]), qc.num_qubits-1-int(qubits[i][1])])
            elif gates[i] == 'ry':
                qml.RY(float(state_params[i]), wires=qc.num_qubits-1-int(qubits[i][0]))
            elif gates[i] == 'rx':
                qml.RY(float(state_params[i]), wires=qc.num_qubits-1-int(qubits[i][0]))

            U(params, wires=[0, 7])
            for i in range(0, 8, 2):
                U(params, wires=[i, i + 1])
            for i in range(1, 7, 2):
                U(params, wires=[i, i + 1])
            U(params, wires=[0, 6])
            U(params, wires=[0, 2])
            U(params, wires=[4, 6])
            U(params, wires=[2, 4])
            
            U(params, wires=[0,4])
            
            result = qml.probs(wires=4)
        return result
    return prep_circuit(gates, params, qubits)

In [8]:
def cross_entropy(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        c_entropy = l * (anp.log(p[l])) + (1 - l) * anp.log(1 - p[1 - l])
        loss = loss + c_entropy
    return -1 * loss

def cost(v, X, Y, U, U_params, circuit):
    pred = [QCNN(x, U, U_params) for x in X]
    loss = cross_entropy(Y, pred)

In [9]:
def train_circuit(X_train, y_train, U, U_params, circuit):
    params = np.random.randn(3*U_params, requires_grad=True)
    opt = qml.NesterovMomentumOptimizer(stepsize=0.01)
    loss_history = []
    steps = 200
    batch_size = 25
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, circuit),
                                                     params)
        loss_history.append(cost_new)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return loss_history, params